In [3]:
import pandas as pd
from lib.cusum import most_change_against_window
from ebmdatalab import bq 

In [4]:
sql = """
SELECT
  pct_id,
  percentile
FROM measures.ccg_data_opioidper1000
"""
df = bq.cached_read(sql, csv_path='../data/ccg_percentiles.zip')

In [5]:
for pct_id, grouped in df.groupby("pct_id"):
    changes = most_change_against_window(grouped.percentile, window=12)
    if changes['improvements']:
        print(pct_id, changes['improvements'])

06Q [{'from': 0.47938596491228075, 'to': 0.4052631578947368, 'period': 1}]
09H [{'from': 0.26228070175438595, 'to': 0.21578947368421053, 'period': 1}]


/home/app/notebook/lib/cusum.py:175: RuntimeWarning: Mean of empty slice
  self.target_means.append(np.nanmean(window))
/home/app/.pyenv/versions/3.8.1/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [6]:
sql = """
SELECT
  practice_id,
  percentile
FROM measures.practice_data_opioidper1000
"""
df2 = bq.cached_read(sql, csv_path='../data/practice_percentiles.zip')

Downloading: 100%|██████████| 492240/492240 [00:18<00:00, 26499.69rows/s]


In [7]:
practice_df = pd.DataFrame(columns=["practice_id", "from", "to", "period"])
for practice_id, grouped in df2.groupby("practice_id"):
    changes = most_change_against_window(grouped.percentile, window=24)
    improvements = changes['improvements'] and changes['improvements'][0] or {}
    if improvements.get("to", 0) > 0:
        practice_df = practice_df.append([{ 'practice_id': practice_id, 'from': improvements["from"], 'to': improvements["to"], 'period': improvements["period"]}])

In [8]:
#https://openprescribing.net/measure/opioidper1000/practice/{}/"
practice_df['delta'] = practice_df["from"] - practice_df["to"]
practice_df = practice_df.sort_values("delta", ascending=False)
practice_df[practice_df.period > 1]

,practice_id,from,to,period,delta
0,Y02520,0.549396,0.291758,2,0.257638
0,D83615,0.824151,0.600787,2,0.223364
0,K81651,0.459473,0.268726,4,0.190747
0,B86050,0.759836,0.666160,2,0.093676
0,P87026,0.792776,0.704068,2,0.088708
0,L81120,0.712973,0.637741,2,0.075231
0,P86602,0.865327,0.801982,3,0.063345
0,A89004,0.930529,0.905139,2,0.025389
